<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/Marfida-Alexander-folder/Marfida%20Alexander/A_Marfida_stajirovka_profpatolog_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>